In [1]:
#import tuxml
import pandas as pd 
import warnings
warnings.filterwarnings('ignore')

#df = tuxml.load_dataset()
# path = r"x264/sampledConfigurations_distBased_t3.csv"
#path = r"sampledConfigurations_distBased_t3.csv"
samples_config = ['distBased', 'divDistBased','henard', 'random', 'solverBased', 'twise']

# for config_name in samples_config:
#     path = r"sampledConfigurations_"+config_name+"_t3.csv"
# #     df.append(pd.read_csv(path, sep=';'))
#     display(path)

# df = pd.read_csv(path, sep=';')


In [2]:
# df_importance = pd.read_csv("feature_importanceRF.csv")
# df_importance = df[df_importance[:100]["Unnamed: 0"].values]
# df_importance.head()

In [2]:
#size_methods = ["vmlinux", "GZIP-bzImage", "GZIP-vmlinux", "GZIP", "BZIP2-bzImage", 
#              "BZIP2-vmlinux", "BZIP2", "LZMA-bzImage", "LZMA-vmlinux", "LZMA", "XZ-bzImage", "XZ-vmlinux", "XZ", 
#?              "LZO-bzImage", "LZO-vmlinux", "LZO", "LZ4-bzImage", "LZ4-vmlinux", "LZ4"]

size_methods = ["Performance"]
#additional_ft = ["nbyes", "nbno", "nbmodule", "nbyesmodule"]

In [3]:
def features_engine(df):
    
    engine_values = []
    for i, j in df.iterrows():
        num1s = 0
        num1s = j.value_counts()[1]
        engine_values.append(num1s)
    
    df['construction'] = engine_values
    

In [4]:
path = r"sampledConfigurations_distBased_t3.csv"
df = pd.read_csv(path, sep=';')
features_engine(df)
display(df)

,root,no_asm,no_8x8dct,no_cabac,no_deblock,no_fast_pskip,no_mbtree,no_mixed_refs,no_weightb,rc_lookahead,rc_lookahead_20,rc_lookahead_40,rc_lookahead_60,ref,ref_1,ref_5,ref_9,Performance,construction
0,1,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,268.720000,6
1,1,1,1,1,1,1,1,0,0,1,0,0,1,1,0,0,1,775.810000,11
2,1,1,1,1,1,1,1,1,0,1,0,0,1,1,0,0,1,644.160000,12
3,1,1,1,1,1,1,1,1,1,1,0,0,1,1,0,0,1,644.033333,13
4,1,1,0,0,0,0,0,0,0,1,0,1,0,1,0,0,1,612.970000,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,1,1,1,1,1,0,0,0,0,1,0,0,1,1,0,0,1,584.976667,9
208,1,1,1,1,1,1,0,0,0,1,0,0,1,1,0,0,1,707.596667,10
209,1,1,1,0,1,0,0,0,1,1,0,0,1,1,0,0,1,590.836667,9
210,1,1,1,0,1,1,0,0,1,1,0,0,1,1,0,0,1,715.583333,10


In [5]:
#learning a model by using linear regression
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import ElasticNet

def run_regressorML(reg, test_size, size_target, x_train, x_test):
    assert(size_target in size_methods)
    reg.fit(x_train, y_train)
    y_pred = reg.predict(x_test)
    #y_pred = reg.intercept_ + np.sum(reg.coef_ * x_test.values, axis=1)
    dfErrors = pd.DataFrame({'Actual':y_test, 'Predicted':y_pred, "error":(y_pred - y_test).abs(), "% error":((y_pred - y_test)/y_test).abs()*100})
    return dfErrors["% error"].describe()

In [6]:
def ft_importances(coef_, col):
    importanceSeries = pd.Series(coef_, index=col.values)
    return importanceSeries[importanceSeries != 0].abs().sort_values(ascending = False)

In [7]:
df_result  = pd.DataFrame(columns = ['algorithm_name', 'ft_dummyfication', 'origin_ft_selection', 'ft_selection', 'hyperparameters', 'size_target', 'test_size', 'coef_order', 'accuracy'])
#df_result  = pd.DataFrame(columns = ['algorithm_name', 'hyperparameters', 'size_target', 'additional_ft', 'test_size', 'coef_order', 'accuracy'])

In [8]:
df_result

,algorithm_name,ft_dummyfication,origin_ft_selection,ft_selection,hyperparameters,size_target,test_size,coef_order,accuracy


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import ElasticNet
from sklearn import linear_model
from sklearn import metrics

ft_importance_enable = True
# def runFeatureSelection(df, config_name, num_features): 
for config_name in samples_config:
    for samplePerc in [0.3,0.4,0.5,0.6]:
        for t in [1,2,3]:
            
            path_name = config_name+"_t"+str(t)
            path = r"sampledConfigurations_"+path_name+".csv"
            
            df = pd.read_csv(path, sep=';')
            num_features = round(samplePerc*len(df.columns))

            df_importance = pd.read_csv("feature_importance_"+path_name+".csv")
            df_importance = df[df_importance[:num_features]["Unnamed: 0"].values]
            
            print("DF with Feature Importance")
            print("Sample features number: "+ str(num_features))
            display(df_importance)
            df_importance.head()

            target = "Performance"
            paramsLasso = {'alpha': [25,10,4,2,1.0,0.8,0.5,0.3,0.2,0.1,0.05,0.02,0.01],
                           'max_iter': [1, 5, 10, 100, 1000],}
            paramsRidge = {'alpha': [25,10,4,2,1.0,0.8,0.5,0.3,0.2,0.1,0.05,0.02,0.01]}
            paramsENet = {'alpha': [25,10,4,2,1.0,0.8,0.5,0.3,0.2,0.1,0.05,0.02,0.01],
                          'l1_ratio': [0.0, 0.5, 1.0, 0.1],
                          'max_iter': [1, 5, 10, 100, 1000],}

            for ft_dummyfication in [False, True]:
                if ft_dummyfication:
                    #Creating dummy variables in pandas
                    df = pd.get_dummies(df, columns=list(df.drop(columns=size_methods).columns.values))
#                     df_importance = pd.get_dummies(df_importance, columns=list(df_importance.drop(columns=size_methods).columns.values))
                    df_importance = pd.get_dummies(df_importance, columns=list(df_importance.columns.values))

                if ft_importance_enable:
                    ft_selection =  {'None': df.drop(columns=size_methods).columns,
                                'RandomForest': df_importance.columns[:num_features],}
                    
                else:
                    ft_selection =  {'None': df.drop(columns=size_methods).columns,
                                     'RandomForest': df.columns[:num_features],}

                for origin_ft_selection in ft_selection:
                    for size in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]:

                        x_train, x_test, y_train, y_test = train_test_split(df[ft_selection[origin_ft_selection]], df[target], test_size=size, random_state=42)  
                        x_train, y_train = np.array(x_train.values), np.array(y_train.values)
                        
                        print(x_train)
                        print(y_train)
                        
                        if len(x_train) > 3:

                            model = {'LinearRegression': LinearRegression(),
                                     'Lasso': GridSearchCV(linear_model.Lasso(), param_grid=paramsLasso, cv=3).fit(x_train, y_train).best_estimator_,
                                     'Ridge': GridSearchCV(linear_model.Ridge(), param_grid=paramsRidge, cv=3).fit(x_train, y_train).best_estimator_,
                                     'ElasticNet': GridSearchCV(ElasticNet(), param_grid=paramsENet, scoring='r2', cv=3).fit(x_train, y_train).best_estimator_,
                                     'PolynomialRegression': LinearRegression()}
                        else:
                            if len(x_train) >= 2:
                                model = {'LinearRegression': LinearRegression(),
                                         'Lasso': GridSearchCV(linear_model.Lasso(), param_grid=paramsLasso, cv=2).fit(x_train, y_train).best_estimator_,
                                         'Ridge': GridSearchCV(linear_model.Ridge(), param_grid=paramsRidge, cv=2).fit(x_train, y_train).best_estimator_,
                                         'ElasticNet': GridSearchCV(ElasticNet(), param_grid=paramsENet, scoring='r2', cv=2).fit(x_train, y_train).best_estimator_,
                                         'PolynomialRegression': LinearRegression()}
                        for key in model:
                            if (key == 'PolynomialRegression') and (origin_ft_selection == 'None'):
                                print("It doesn't scale")
                            else:
                                if (key == 'PolynomialRegression'):
                                    #for 2 options we have: 𝑓(𝑥₁, 𝑥₂) = 𝑏₀ + 𝑏₁𝑥₁ + 𝑏₂𝑥₂ + 𝑏₃𝑥₁² + 𝑏₄𝑥₁𝑥₂ + 𝑏₅𝑥₂²
                                    x_train = PolynomialFeatures(degree=2, include_bias=False).fit_transform(x_train)
                                    x_test = PolynomialFeatures(degree=2, include_bias=False).fit_transform(x_test)

                                res_model = run_regressorML(model[key], test_size=size, size_target=target, x_train=x_train, x_test=x_test)

                                if (key == 'PolynomialRegression'):
                                    model[key].coef_ = model[key].coef_[:num_features]

#                                 print(model[key].coef_)
                                coef_order = ft_importances(model[key].coef_, col=ft_selection[origin_ft_selection])
                                df_result.loc[len(df_result)] = [key, ft_dummyfication, origin_ft_selection, ft_selection[origin_ft_selection], model[key], target, size, coef_order, res_model]
            display(df_result)
            if ft_importance_enable:
                df_result.to_csv("results_with_feature_importance_"+path_name+"_"+str(num_features)+".csv", header=True)
            else:
                df_result.to_csv("results_"+path_name+"_"+str(num_features)+".csv", header=True)

DF with Feature Importance
Sample features number: 5


,ref_1,ref_9,ref_5,rc_lookahead_60,no_fast_pskip
0,1,0,0,0,0
1,0,1,0,1,1
2,1,0,0,0,0
3,1,0,0,1,0
4,1,0,0,1,1
5,1,0,0,1,1
6,0,0,1,0,0
7,0,0,1,0,1
8,0,0,1,1,1
9,1,0,0,0,0


[[1 1 0 0 0 0 0 0 0 1 1 0 0 1 1 0 0]
 [1 1 1 1 1 1 1 1 0 1 0 0 1 1 0 1 0]
 [1 1 1 1 1 1 1 1 0 1 0 0 1 1 1 0 0]
 [1 1 1 1 0 0 0 0 0 1 0 1 0 1 1 0 0]
 [1 1 1 1 1 1 1 0 0 1 0 0 1 1 0 0 1]
 [1 1 1 1 1 0 0 0 0 1 1 0 0 1 0 0 1]
 [1 1 1 1 1 1 1 0 0 1 0 0 1 1 1 0 0]
 [1 1 1 1 1 1 0 0 0 1 0 1 0 1 0 1 0]
 [1 1 1 1 0 0 0 0 0 1 0 0 1 1 1 0 0]
 [1 1 1 1 0 0 0 0 0 1 0 1 0 1 0 1 0]]
[268.72       455.72       279.13       246.48666667 775.81
 581.40666667 279.22333333 487.37333333 247.73       418.56333333]
It doesn't scale
[[1 1 1 1 1 1 1 1 0 1 0 0 1 1 0 1 0]
 [1 1 1 1 1 1 1 1 0 1 0 0 1 1 1 0 0]
 [1 1 1 1 0 0 0 0 0 1 0 1 0 1 1 0 0]
 [1 1 1 1 1 1 1 0 0 1 0 0 1 1 0 0 1]
 [1 1 1 1 1 0 0 0 0 1 1 0 0 1 0 0 1]
 [1 1 1 1 1 1 1 0 0 1 0 0 1 1 1 0 0]
 [1 1 1 1 1 1 0 0 0 1 0 1 0 1 0 1 0]
 [1 1 1 1 0 0 0 0 0 1 0 0 1 1 1 0 0]
 [1 1 1 1 0 0 0 0 0 1 0 1 0 1 0 1 0]]
[455.72       279.13       246.48666667 775.81       581.40666667
 279.22333333 487.37333333 247.73       418.56333333]
It doesn't scale
[[1 1 1 1 1 1 

[[0 1 1 0 1]
 [0 1 1 0 1]
 [1 0 0 1 1]
 [1 0 0 1 1]
 [0 1 1 0 1]
 [1 0 1 0 0]
 [0 1 1 0 1]
 [1 0 1 0 0]]
[279.13       246.48666667 775.81       581.40666667 279.22333333
 487.37333333 247.73       418.56333333]
[[0 1 1 0 1]
 [1 0 0 1 1]
 [1 0 0 1 1]
 [0 1 1 0 1]
 [1 0 1 0 0]
 [0 1 1 0 1]
 [1 0 1 0 0]]
[246.48666667 775.81       581.40666667 279.22333333 487.37333333
 247.73       418.56333333]
[[1 0 0 1 1]
 [1 0 0 1 1]
 [0 1 1 0 1]
 [1 0 1 0 0]
 [0 1 1 0 1]
 [1 0 1 0 0]]
[775.81       581.40666667 279.22333333 487.37333333 247.73
 418.56333333]
[[0 1 1 0 1]
 [1 0 1 0 0]
 [0 1 1 0 1]
 [1 0 1 0 0]]
[279.22333333 487.37333333 247.73       418.56333333]
[[1 0 1 0 0]
 [0 1 1 0 1]
 [1 0 1 0 0]]
[487.37333333 247.73       418.56333333]
[[0 1 1 0 1]
 [1 0 1 0 0]]
[247.73       418.56333333]
[[1 0 1 0 0]]
[418.56333333]


,algorithm_name,ft_dummyfication,origin_ft_selection,ft_selection,hyperparameters,size_target,test_size,coef_order,accuracy
0,LinearRegression,False,None,"Index(['root', 'no_asm', 'no_8x8dct', 'no_caba...",LinearRegression(),Performance,0.1,ref_9 2.729467e+02 ref_1 ...,count 2.000000 mean 46.050138 std ...
1,Lasso,False,None,"Index(['root', 'no_asm', 'no_8x8dct', 'no_caba...","Lasso(alpha=10, max_iter=10)",Performance,0.1,ref_9 179.815357 ref_1 1...,count 2.000000 mean 6.616934 std ...
2,Ridge,False,None,"Index(['root', 'no_asm', 'no_8x8dct', 'no_caba...",Ridge(alpha=0.5),Performance,0.1,ref_9 169.613133 ref_1 ...,count 2.000000 mean 3.465232 std 1...
3,ElasticNet,False,None,"Index(['root', 'no_asm', 'no_8x8dct', 'no_caba...","ElasticNet(alpha=10, l1_ratio=1.0, max_iter=10)",Performance,0.1,ref_9 179.815357 ref_1 1...,count 2.000000 mean 6.616934 std ...
4,LinearRegression,False,None,"Index(['root', 'no_asm', 'no_8x8dct', 'no_caba...",LinearRegression(),Performance,0.2,ref_9 2.729467e+02 ref_1 ...,count 3.000000 mean 53.595418 std ...
...,...,...,...,...,...,...,...,...,...
157,LinearRegression,True,RandomForest,"Index(['ref_1_0', 'ref_1_1', 'ref_9_0', 'ref_9...",LinearRegression(),Performance,0.9,"Series([], dtype: float64)",count 11.000000 mean 42.748672 std ...
158,Lasso,True,RandomForest,"Index(['ref_1_0', 'ref_1_1', 'ref_9_0', 'ref_9...","Lasso(alpha=25, max_iter=1)",Performance,0.9,"Series([], dtype: float64)",count 11.000000 mean 42.748672 std ...
159,Ridge,True,RandomForest,"Index(['ref_1_0', 'ref_1_1', 'ref_9_0', 'ref_9...",Ridge(alpha=25),Performance,0.9,"Series([], dtype: float64)",count 11.000000 mean 42.748672 std ...
160,ElasticNet,True,RandomForest,"Index(['ref_1_0', 'ref_1_1', 'ref_9_0', 'ref_9...","ElasticNet(alpha=25, l1_ratio=0.0, max_iter=1)",Performance,0.9,"Series([], dtype: float64)",count 11.000000 mean 42.748672 std ...


...

[[1 1 1 1 0 0 0 0 0 1 0 1 0 1 1 0 0]
 [1 1 1 1 1 0 0 0 0 1 0 0 1 1 0 0 1]
 [1 1 1 1 1 1 0 0 0 1 0 1 0 1 0 1 0]
 [1 1 1 1 0 1 1 1 1 1 0 1 0 1 0 1 0]
 [1 1 1 1 1 1 1 1 1 1 0 1 0 1 0 0 1]
 [1 1 0 0 0 0 0 0 0 1 1 0 0 1 0 1 0]
 [1 1 1 1 1 0 0 0 0 1 0 1 0 1 1 0 0]
 [1 1 1 1 1 1 1 1 1 1 0 1 0 1 0 1 0]
 [1 1 0 0 0 0 0 0 0 1 0 1 0 1 0 1 0]
 [1 1 0 0 0 0 0 0 0 1 0 1 0 1 0 0 1]
 [1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 0 0]
 [1 1 1 1 1 1 1 1 0 1 0 0 1 1 0 1 0]
 [1 1 1 1 1 1 1 1 0 1 0 1 0 1 0 0 1]
 [1 1 1 1 1 0 0 0 0 1 0 1 0 1 0 0 1]
 [1 1 1 1 0 0 0 0 0 1 0 0 1 1 0 0 1]
 [1 1 1 1 0 0 0 0 0 1 0 1 0 1 0 0 1]
 [1 1 1 1 1 0 0 0 0 1 1 0 0 1 0 1 0]
 [1 1 1 1 0 0 0 0 0 1 0 0 1 1 0 1 0]
 [1 1 1 0 0 0 0 0 0 1 0 0 1 1 1 0 0]
 [1 1 1 1 0 0 0 0 0 1 1 0 0 1 0 0 1]
 [1 1 1 1 1 1 1 1 0 1 1 0 0 1 0 0 1]
 [1 1 1 1 1 0 0 0 0 1 1 0 0 1 0 0 1]
 [1 1 1 1 1 1 1 0 0 1 1 0 0 1 0 1 0]
 [1 1 1 1 1 1 1 0 0 1 0 1 0 1 0 1 0]
 [1 1 1 0 0 0 0 0 0 1 1 0 0 1 0 1 0]
 [1 1 1 0 0 0 0 0 0 1 0 0 1 1 0 0 1]
 [1 1 1 1 1 1 1 0 0 1 0 1 0 1 1 0 0]
 

It doesn't scale
[[1 1 1 1 1 1 1 0 0 1 0 1 0 1 1 0 0]
 [1 1 1 0 0 0 0 0 0 1 0 1 0 1 0 1 0]
 [1 1 0 1 1 1 1 1 1 1 0 1 0 1 0 1 0]
 [1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 0 0]
 [1 1 1 1 1 1 0 0 0 1 1 0 0 1 1 0 0]
 [1 1 0 0 0 0 0 0 0 1 0 1 0 1 1 0 0]
 [1 1 1 1 1 1 1 1 0 1 0 1 0 1 0 1 0]
 [1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 0 1]
 [1 1 1 1 1 0 0 0 0 1 1 0 0 1 1 0 0]
 [1 1 1 1 1 0 0 0 0 1 0 0 1 1 0 1 0]
 [1 1 1 1 1 0 0 0 0 1 0 1 0 1 0 1 0]
 [1 1 1 1 0 0 0 0 0 1 1 0 0 1 1 0 0]
 [1 1 1 1 1 1 0 0 0 1 1 0 0 1 0 1 0]
 [1 1 1 1 1 1 1 0 0 1 0 0 1 1 0 0 1]
 [1 1 1 1 1 1 0 0 0 1 0 0 1 1 1 0 0]
 [1 1 1 1 1 1 1 1 0 1 0 0 1 1 0 0 1]
 [1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 1 0]
 [1 1 1 1 1 1 1 0 0 1 0 0 1 1 0 1 0]
 [1 1 1 0 0 0 0 0 0 1 0 0 1 1 0 1 0]
 [1 1 1 1 1 1 1 1 0 1 0 0 1 1 1 0 0]
 [1 1 1 0 0 0 0 0 0 1 1 0 0 1 0 0 1]
 [1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0]
 [1 1 1 1 1 1 1 0 0 1 0 0 1 1 1 0 0]
 [1 1 0 0 0 0 0 0 0 1 0 0 1 1 1 0 0]
 [1 1 1 1 1 1 1 0 0 1 1 0 0 1 0 0 1]
 [1 1 1 1 1 1 0 0 0 1 0 0 1 1 0 1 0]
 [1 1 1 1 0 0 0 0 0 1

[[0 1 0 0 0]
 [0 1 0 1 1]
 [0 1 0 0 0]
 [0 0 1 1 0]
 [0 0 1 1 0]
 [0 0 1 0 0]
 [0 1 0 0 0]
 [1 0 0 1 0]
 [0 0 1 0 0]
 [0 0 1 1 1]
 [1 0 0 1 1]
 [1 0 0 1 0]
 [1 0 0 0 0]
 [0 0 1 1 1]
 [0 1 0 1 1]
 [1 0 0 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [1 0 0 0 0]
 [0 0 1 1 0]
 [0 1 0 1 0]
 [1 0 0 1 0]
 [0 1 0 1 1]
 [0 0 1 1 1]
 [0 0 1 1 0]
 [0 0 1 0 0]
 [1 0 0 1 1]
 [0 1 0 0 0]
 [1 0 0 1 1]
 [1 0 0 1 0]
 [1 0 0 0 0]
 [0 1 0 1 0]
 [0 0 1 1 0]
 [1 0 0 0 0]
 [0 1 0 1 0]
 [1 0 0 0 0]
 [1 0 0 1 0]
 [0 0 1 0 0]
 [0 0 1 1 1]]
[579.17666667 644.15666667 581.40666667 537.15333333 537.14333333
 423.07666667 588.54       279.24666667 425.15       476.12666667
 279.24666667 249.36333333 270.65666667 455.68       643.96
 244.23       421.11333333 420.01       244.39       485.53666667
 775.81       252.82666667 644.16       455.49333333 537.11333333
 426.41666667 279.13       585.35       279.26666667 279.22333333
 271.96666667 775.85333333 488.77333333 247.73       775.66333333
 254.58666667 251.57333333 418.56333

It doesn't scale
[[1 1 0 1 0 1 0 1 0 1 0 1 0 1 1 0 1 1 0 0 1 1 0 1 1 0 0 1 1 0]
 [1 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 1 0 1 1 0 1 0 1 1 0 1 0 0 1]
 [1 1 0 1 0 1 0 1 1 0 1 0 1 0 1 0 1 0 1 1 0 1 0 1 0 1 1 0 1 0]
 [1 1 0 1 0 1 0 1 1 0 1 0 1 0 1 0 1 1 0 1 0 0 1 1 1 0 0 1 1 0]
 [1 1 0 1 0 1 0 1 1 0 1 0 1 0 1 0 1 1 0 0 1 1 0 1 1 0 0 1 1 0]
 [1 1 0 1 0 1 1 0 1 0 1 0 1 0 1 0 1 0 1 1 0 1 0 1 0 1 1 0 1 0]
 [1 1 0 1 0 1 0 1 0 1 1 0 1 0 1 0 1 0 1 1 0 1 0 1 1 0 0 1 1 0]
 [1 1 0 1 0 1 0 1 0 1 0 1 1 0 1 0 1 1 0 1 0 0 1 1 1 0 1 0 0 1]
 [1 1 0 1 0 1 0 1 0 1 1 0 1 0 1 0 1 1 0 1 0 0 1 1 0 1 1 0 1 0]
 [1 1 0 1 0 1 0 1 0 1 0 1 0 1 1 0 1 1 0 1 0 0 1 1 1 0 1 0 0 1]
 [1 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 1 0 1 1 0 1 0 1 1 0 0 1 1 0]
 [1 1 0 1 0 1 0 1 0 1 0 1 1 0 1 0 1 1 0 1 0 0 1 1 1 0 0 1 1 0]
 [1 1 0 1 1 0 1 0 1 0 1 0 1 0 1 0 1 1 0 1 0 0 1 1 1 0 0 1 1 0]
 [1 1 0 1 0 1 0 1 0 1 0 1 0 1 1 0 1 1 0 1 0 0 1 1 0 1 1 0 1 0]
 [1 1 0 1 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 1 0 1 0 1 1 0 1 0 0 1]
 [1 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 1 0 

[[1 0 0 1 1]
 [1 0 0 1 1]
 [1 0 0 1 1]
 [1 0 1 0 0]
 [1 0 1 0 0]
 [1 0 1 0 0]
 [1 0 0 1 1]
 [0 1 1 0 1]
 [1 0 1 0 0]
 [1 0 1 0 0]
 [0 1 1 0 1]
 [0 1 1 0 1]
 [0 1 1 0 1]
 [1 0 1 0 0]
 [1 0 0 1 1]
 [0 1 1 0 1]
 [1 0 1 0 0]
 [1 0 1 0 0]
 [0 1 1 0 1]
 [1 0 1 0 0]
 [1 0 0 1 1]
 [0 1 1 0 1]
 [1 0 0 1 1]
 [1 0 1 0 0]
 [1 0 1 0 0]
 [1 0 1 0 0]
 [0 1 1 0 1]
 [1 0 0 1 1]
 [0 1 1 0 1]
 [0 1 1 0 1]
 [0 1 1 0 1]
 [1 0 0 1 1]
 [1 0 1 0 0]
 [0 1 1 0 1]
 [1 0 0 1 1]
 [0 1 1 0 1]
 [0 1 1 0 1]
 [1 0 1 0 0]
 [1 0 1 0 0]]
[579.17666667 644.15666667 581.40666667 537.15333333 537.14333333
 423.07666667 588.54       279.24666667 425.15       476.12666667
 279.24666667 249.36333333 270.65666667 455.68       643.96
 244.23       421.11333333 420.01       244.39       485.53666667
 775.81       252.82666667 644.16       455.49333333 537.11333333
 426.41666667 279.13       585.35       279.26666667 279.22333333
 271.96666667 775.85333333 488.77333333 247.73       775.66333333
 254.58666667 251.57333333 418.56333

In [ ]:
len(x_train)

In [ ]:
df_result['accuracy'][0]

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

rs = []
for i in range(len(df_result['accuracy'])):
    rs.append(df_result['accuracy'][i]['mean'])

# rs
# display (df_result['algorithm_name'], df_result['accuracy'], rs)

plt.scatter(df_result['algorithm_name'], rs, color ="blue", marker = "s")
plt.plot(df_result['algorithm_name'], rs, color ="blue")


In [ ]:
# Boxplot
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


data = [model['LinearRegression'].coef_, model['Lasso'].coef_, model['Ridge'].coef_, model['ElasticNet'].coef_, model['PolynomialRegression'].coef_]
fig1, ax1 = plt.subplots()
ax1.set_title('Coeficient of feature importance for all algorithms')
ax1.boxplot(data)
plt.show()